# Deutsch-Jozsa Algorithm: Constant vs Balanced for n-bit Inputs
# By Vivek Rajagopalan | Date: 21Dec2025

---
### **What We’ll Learn**
Extend Deutsch’s algorithm to check if an `n`-bit function is constant or balanced. Quantum still needs 1 query; classical could need up to `2^(n-1) + 1`!

---
### **Problem Setup**
Function `f: {0,1}^n → {0,1}` (takes n bits, returns 0/1). We need to confirm:
- **Constant**: All inputs → same output (all 0s or all 1s).
- **Balanced**: Exactly half of inputs → 0, half → 1.

---
### **Step 1: Classical vs Quantum Intuition**
Classical: To confirm balanced, you might test inputs until you find two with different outputs (worst case: `2^(n-1) + 1` queries).
Quantum: Use superposition to test *all inputs at once* in 1 query!


---
### **Step 2: Quantum Circuit for Deutsch-Jozsa**
Let’s build the circuit for `n=2` (easy to visualize).

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram

def deutsch_jozsa_oracle(s, n):
    # s is the hidden string (for testing; in real problem, s is unknown)
    # Oracle computes f(x) = x·s mod 2 (but here, we'll use s to decide balanced/constant)
    # For our example, let's make f balanced if s is non-zero, else constant
    qc = QuantumCircuit(n+1)
    for i in range(n):
        if s[i] == '1':
            qc.cx(i, n)  # CX gate: flip output qubit if input qubit i is 1
    return qc

def quantum_deutsch_jozsa(n, s):
    # Create circuit with n input qubits, 1 output qubit, and n classical bits
    qc = QuantumCircuit(n+1, n)

    # Step 1: Initialize all qubits to superposition
    for i in range(n+1):
        qc.h(i)

    # Step 2: Add oracle (black box)
    oracle = deutsch_jozsa_oracle(s, n)
    qc = qc.compose(oracle)

    # Step 3: Apply Hadamard to input qubits again, then measure
    for i in range(n):
        qc.h(i)
        qc.measure(i, i)

    return qc

In [ ]:
# Let's test with n=2
n = 2

# Example 1: Constant function (s=00)
s_constant = '00'
qc_dj_constant = quantum_deutsch_jozsa(n, s_constant)
print("Circuit (s=00, Constant):")
qc_dj_constant.draw(output='mpl')

simulator = Aer.get_backend('qasm_simulator')
result_constant = execute(qc_dj_constant, simulator, shots=1024).result()
counts_constant = result_constant.get_counts()
print("Measurement Results (Constant):")
plot_histogram(counts_constant)

In [ ]:
# Example 2: Balanced function (s=10)
s_balanced = '10'
qc_dj_balanced = quantum_deutsch_jozsa(n, s_balanced)
print("Circuit (s=10, Balanced):")
qc_dj_balanced.draw(output='mpl')

result_balanced = execute(qc_dj_balanced, simulator, shots=1024).result()
counts_balanced = result_balanced.get_counts()
print("Measurement Results (Balanced):")
plot_histogram(counts_balanced)